In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> Data Preprocessing

In [ ]:
train_df = pd.read_csv("../input/santander-customer-satisfaction/train.csv",encoding='latin-1')
print('dataset shape:', train_df.shape)
train_df.head(3)

In [ ]:
test_df = pd.read_csv("../input/santander-customer-satisfaction/test.csv",encoding='latin-1')
print('dataset shape:', test_df.shape)
test_df.head(3)

In [ ]:
train_df.info()

In [ ]:
print(train_df['TARGET'].value_counts())

unsatisfied_cnt = train_df[train_df['TARGET'] == 1]['TARGET'].count()
total_cnt = train_df['TARGET'].count()

print('unsatisfied Ratio {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

In [ ]:
train_df.describe( )

In [ ]:
print(train_df['var3'].value_counts( )[:10])

In [ ]:
print(test_df['var3'].value_counts( )[:10])

> -999999 is a NaN -> Should replace or Drop it 

In [ ]:
# var3 value replace -999999 to 2, Drop ID feature
train_df['var3'].replace(-999999, 2, inplace=True)
train_df.drop('ID',axis=1 , inplace=True)

test_df['var3'].replace(-999999, 2, inplace=True)
test_df.drop('ID',axis=1 , inplace=True)

# Split feature, lable. 
X_features = train_df.iloc[:, :-1]
y_labels = train_df.iloc[:, -1]
print('Feature data shape:{0}'.format(X_features.shape))

X_test = test_df

In [ ]:
from sklearn.model_selection import train_test_split
# Split train, validation set 
X_train, X_val, y_train, y_val = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0)
train_cnt = y_train.count()
val_cnt = y_val.count()
print('train set Shape:{0}, val set Shape:{1}'.format(X_train.shape , X_val.shape))

print(' ratio of train set label')
print(y_train.value_counts()/train_cnt)
print('\n ratio of validation set label')
print(y_val.value_counts()/val_cnt)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# n_estimators: 500 
xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

# evaluation: auc, early_stopping_roubnds: 100. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
            eval_metric="auc", eval_set=[(X_train, y_train), (X_val, y_val)])

xgb_roc_score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
# n_estimators: 1000, learning_rate=0.02, reg_alpha=0.03. 
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=7,\
                        min_child_weight=1, colsample_bytree=0.75, reg_alpha=0.03)

# evaluation metric: auc, early stopping: 200  
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, 
            eval_metric="auc",eval_set=[(X_train, y_train), (X_val, y_val)])

xgb_roc_score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1,1,figsize=(10,8))
plot_importance(xgb_clf, ax=ax , max_num_features=20,height=0.4)

In [ ]:
submission = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv')
submission.head()
#finals_pred = xgb_clf.predict(X_test)

#finals_pred

In [ ]:
target = xgb_clf.predict(X_test)

submission['TARGET'] = target

In [ ]:
submission.to_csv('submission.csv', index=False)